In [1]:
import functools

import clingo

from typing import Sequence

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' ', report_answersets=True):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            if report_answersets:
                print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        solve_result = solve_handle.get()
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(solve_result, "{}{}".format(len(models), cardinality_suffix))

    return models
run = functools.partial(solve, report_answersets=False)

In [3]:
def check(programs: Sequence[str], consistency_checks:str, grounding_context=None):
    print("Run without checks:")
    wo_checks = solve(programs, grounding_context, report_answersets=False)
    print("Run with checks:")
    w_checks = solve((*programs, consistency_checks), grounding_context, report_answersets=False)
    return len(wo_checks) == len(w_checks)

In [4]:
static_physics = """

occurs(object(pipeline, A), change(flows, ()), T) :-
  observe(object(pipeline, A), value(liquidLevel, full), T),
  observe(object(pipeline, A), value(connected, object(pipeline, B)), T),
  observe(object(valve, V), value(controls, object(pipeline, A)), T),
  occurs(object(valve, V), change(position, close), T).

occurs(object(pipeline, A), change(flows, object(pipeline, B)), T) :-
  observe(object(pipeline, A), value(liquidLevel, full), T),
  observe(object(pipeline, A), value(connected, object(pipeline, B)), T),
  observe(object(valve, V), value(controls, object(pipeline, A)), T),
  occurs(object(valve, V), change(position, open), T).

occurs(object(pipeline, A), change(flows, object(pipeline, B)), T) :-
  observe(object(pipeline, A), value(liquidLevel, full), T),
  observe(object(pipeline, B), value(liquidLevel, empty), T),
  observe(object(pipeline, A), value(connected, object(pipeline, B)), T),
  observe(object(valve, V), value(controls, object(pipeline, A)), T),
  observe(object(valve, V), value(position, open), T).

occurs(object(pipeline, B), change(liquidLevel, full), T) :-
  observe(object(pipeline, B), value(liquidLevel, empty), T),
  occurs(object(pipeline, A), change(flows, object(pipeline, B)), T).

occurs(object(pipeline, B), change(liquidLevel, empty), T) :-
  observe(object(pipeline, B), value(liquidLevel, full), T),
  observe(object(pipeline, A), value(connected, object(pipeline, B)), T),
  occurs(object(pipeline, A), change(flows, ()), T).

"""
run([static_physics]);

SAT 1


<block>:4:3-60: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(liquidLevel,full),T)

<block>:5:3-73: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(connected,object(pipeline,B)),T)

<block>:6:3-69: info: atom does not occur in any rule head:
  observe(object(valve,V),value(controls,object(pipeline,A)),T)

<block>:7:3-55: info: atom does not occur in any rule head:
  occurs(object(valve,V),change(position,close),T)

<block>:10:3-60: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(liquidLevel,full),T)

<block>:11:3-73: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(connected,object(pipeline,B)),T)

<block>:12:3-69: info: atom does not occur in any rule head:
  observe(object(valve,V),value(controls,object(pipeline,A)),T)

<block>:13:3-54: info: atom does not occur in any rule head:
  occurs(object(valve,V),change(position,open),T)

<block>:16:3-60: info: atom doe

In [5]:
action_consequences = """

occurs(object(valve, A), change(position, Position), T) :-
  observe(object(valve, A), value(at, Coordinates), T),
  observe(Who, value(at, Coordinates), T),
  occurs(Who, action(Position, object(valve, A)), T).

"""
run([action_consequences]);

SAT 1


<block>:4:3-55: info: atom does not occur in any rule head:
  observe(object(valve,A),value(at,Coordinates),T)

<block>:5:3-42: info: atom does not occur in any rule head:
  observe(Who,value(at,Coordinates),T)

<block>:6:3-53: info: atom does not occur in any rule head:
  occurs(Who,action(Position,object(valve,A)),T)



In [6]:
static = """

observe(Who, What, ST) :-
  init(Who, What),
  time(ST), not time(ST-1).

observe(Who, value(Property, Value), T+1) :-
  time(T+1),
  observe(Who, value(Property, Value), T),
  not occurs(Who, change(Property, _), T).

observe(Who, value(Property, NewValue), T+1) :-
  time(T+1),
  occurs(Who, change(Property, NewValue), T).

"""
run([static]);

SAT 1


<block>:4:3-18: info: atom does not occur in any rule head:
  init(Who,What)

<block>:5:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:5:17-27: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:8:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:10:7-42: info: atom does not occur in any rule head:
  occurs(#X0,change(#X1,#P2),#X3)

<block>:13:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:14:3-45: info: atom does not occur in any rule head:
  occurs(Who,change(Property,NewValue),T)



In [7]:
instance = """

time(1..5).
init(object(pipeline, source), value(at, (2,3))).
init(object(pipeline, source), value(liquidLevel, full)).
init(object(pipeline, source), value(connected, object(pipeline, connect))).

init(object(valve, source), value(at, (2,3))).
init(object(valve, source), value(position, closed)).
init(object(valve, source), value(controls, object(pipeline, source))).

init(object(pipeline, sink), value(at, (2,1))).
init(object(pipeline, sink), value(liquidLevel, empty)).

init(object(valve, sink), value(at, (2,1))).
init(object(valve, sink), value(position, closed)).
init(object(valve, sink), value(controls, object(pipeline, connect))). % sic! (controls connect)

init(object(pipeline, connect), value(at, (2,2))).
init(object(pipeline, connect), value(liquidLevel, empty)).
init(object(pipeline, connect), value(connected, object(pipeline, sink))).

init(agent("Lieferant"), value(at, (2,3))).
init(agent("Empfänger"), value(at, (2,1))).

occurs(agent("Lieferant"), action(open, object(valve, source)), 1).
occurs(agent("Empfänger"), action(open, object(valve, sink)), 1).

"""
run([instance]);

SAT 1


In [8]:
prg = (
    static_physics,
    static,
    action_consequences,
    instance
)

In [9]:
solve((*prg,
       '#show init/2.',
       '#show occurs/3.',
       '#show observe/3.'
       '#show deontic/3.'), sep='\n');

Answer 1: {
init(agent("Empfänger"),value(at,(2,1)))
init(agent("Lieferant"),value(at,(2,3)))
init(object(pipeline,connect),value(at,(2,2)))
init(object(pipeline,connect),value(connected,object(pipeline,sink)))
init(object(pipeline,connect),value(liquidLevel,empty))
init(object(pipeline,sink),value(at,(2,1)))
init(object(pipeline,sink),value(liquidLevel,empty))
init(object(pipeline,source),value(at,(2,3)))
init(object(pipeline,source),value(connected,object(pipeline,connect)))
init(object(pipeline,source),value(liquidLevel,full))
init(object(valve,sink),value(at,(2,1)))
init(object(valve,sink),value(controls,object(pipeline,connect)))
init(object(valve,sink),value(position,closed))
init(object(valve,source),value(at,(2,3)))
init(object(valve,source),value(controls,object(pipeline,source)))
init(object(valve,source),value(position,closed))
observe(agent("Empfänger"),value(at,(2,1)),1)
observe(agent("Empfänger"),value(at,(2,1)),2)
observe(agent("Empfänger"),value(at,(2,1)),3)
observe(age

<block>:1:17-33: info: no atoms over signature occur in program:
  deontic/3

